# Regional PE Valuation
© 2026 Jim Domeij

This calculates 10-year P/E valuation statistics (average, standard deviation, minimum and maximum) for major equity regions. Current levels are compared to these statistics to assess the relative expansiveness vs history and other regions.

The BQL query builds a shared 10-year date range, defines a reusable forward P/E series, and computes `#avg`, `#std`, `#min`, `#max`, plus the latest value `#last` for each index in the universe. BQL returns a Polars DataFrame with `ID` and those five fields, which are renamed for plotting.

The chart visualizes each region’s distribution using a box for average ±1 standard deviation, whiskers for min/max, a diamond for the average, and a triangle for the latest value.


In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/macro") / "pe-valuations-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.LayerChart(...)

In [2]:
# Get P/E for different regions
from polars_bloomberg import BQuery

query = """
let(
    #series=headline_pe_ratio(dates=range(-10Y, 0D, frq=D));
    #avg=avg(#series);
    #std=std(#series);
    #min=min(#series);
    #max=max(#series);
    #last=headline_pe_ratio();
)
for([
    'SPX Index',
    'B500XM7T Index',
    'SXXP Index',
    'TPX Index',
    'UKX Index',
    'MXEF Index',
    'SHSZ300 Index'
])
get(#avg, #std, #min, #max, #last)
with(fpt=BT, fpo=1)
preferences(dropCols=['REVISION_DATE', 'AS_OF_DATE', 'PERIOD_END_DATE'])
"""

with BQuery() as bq:
    df = bq.bql(query).combine()
df.head()

ID,#avg,#std,#min,#max,#last
str,f64,f64,f64,f64,f64
"""SPX Index""",18.854204,2.229219,13.563658,23.339709,22.050241
"""B500XM7T Index""",17.725393,1.807348,12.613702,21.704093,19.95632
"""SXXP Index""",14.464378,1.579359,10.638401,18.889421,15.256618
"""TPX Index""",14.718192,1.778616,11.116715,20.47531,16.522521
"""UKX Index""",12.791636,1.702829,8.702596,17.191449,13.389673


In [3]:
# Create Chart
import altair as alt
import polars as pl

df_plot = df.rename(
    {
        "#avg": "avg",
        "#std": "std",
        "#min": "min",
        "#max": "max",
        "#last": "last",
    }
).with_columns(
    pl.col("ID")
    .replace(
        {
            "SPX Index": "US",
            "B500XM7T Index": "US ex-Mag7",
            "SXXP Index": "Europe",
            "TPX Index": "Japan",
            "UKX Index": "UK",
            "MXEF Index": "EM",
            "SHSZ300 Index": "China",
        }
    )
    .alias("ID"),
    (pl.col("avg") - pl.col("std")).alias("box_low"),
    (pl.col("avg") + pl.col("std")).alias("box_high"),
)


base = alt.Chart(df_plot).transform_calculate(
    last_label="format(datum.last, '.1f') + 'x'",
).encode(
    x=alt.X("ID:N", title="", sort=alt.SortField("last", order="descending")),
    tooltip=[
        alt.Tooltip("ID:N"),
        alt.Tooltip("avg:Q", title="Avg", format=".1f"),
        alt.Tooltip("std:Q", title="Std", format=".1f"),
        alt.Tooltip("min:Q", title="Min", format=".1f"),
        alt.Tooltip("max:Q", title="Max", format=".1f"),
        alt.Tooltip("last:Q", title="Last", format=".1f"),
    ],
)

whiskers = base.mark_rule(color="grey").encode(
    y="min:Q",
    y2="max:Q",
)

box = base.mark_bar(size=28, color="#cfead6").encode(
    y=alt.Y(
        "box_low:Q",
        title="12M Forward P/E",
        axis=alt.Axis(labelExpr="datum.value + 'x'"),
    ).scale(zero=False),
    y2="box_high:Q",
)

avg_points = base.mark_point(shape="diamond", size=70, color="#2e8b57").encode(
    y="avg:Q",
)

last_points = base.mark_point(
    shape="triangle-up", filled=True, color="#ff6a3d", size=120
).encode(
    y="last:Q",
)
last_text = last_points.mark_text(color="#ff6a3d", dx=20).encode(
    text=alt.Text("last_label:N")
)

chart = alt.layer(whiskers, box, avg_points, last_points, last_text).properties(
    width=600,
    height=400,
)

# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)
